# Jupyter Notebook para Debugging do app.py

Este notebook permite testar e depurar o código do `app.py`. Ele contém o código completo para configurar as chaves da API, definir as funções para fazer as requisições às APIs, e testar o endpoint `/api/compile_responses`.

## 1. Importar as Bibliotecas e Configurar as API Keys

In [ ]:
import os
import json
from flask import Flask, request, jsonify, render_template
from dotenv import load_dotenv
from openai import OpenAI
#from IPython.display import Markdown, display, update_display

# Carregar variáveis de ambiente do ficheiro .env
load_dotenv()

app = Flask(__name__, template_folder='template', static_folder='static')

# Configurar as API Keys
try:
    gemini_api_key = os.environ.get("GEMINI_API_KEY")
    if not gemini_api_key:
        raise ValueError("A GEMINI_API_KEY não foi definida no ficheiro .env")

    food_api_key = os.environ.get("FOOD_API_KEY")
    if not food_api_key:
        raise ValueError("A FOOD_API_KEY não foi definida no ficheiro .env")

    activities_api_key = os.environ.get("ACTIVITIES_API_KEY")
    if not activities_api_key:
        raise ValueError("A ACTIVITIES_API_KEY não foi definida no ficheiro .env")

    spotify_api_key = os.environ.get("SPOTIFY_API_KEY")
    if not spotify_api_key:
        raise ValueError("A SPOTIFY_API_KEY não foi definida no ficheiro .env")

    party_place_api_key = os.environ.get("PARTY_PLACE_API_KEY")
    if not party_place_api_key:
        raise ValueError("A PARTY_PLACE_API_KEY não foi definida no ficheiro .env")
    # Configuração das APIs
    api_configurations = {
        "gemini": {
            "model": "gemini-2.0-flash",  # Ou outro modelo da OpenAI de sua preferência
            "url": "https://generativelanguage.googleapis.com/v1beta/openai/",
            "api_key": gemini_api_key,
            "client": None  # Inicializado posteriormente
        },
        "food": {
            "model": "gemini-2.0-flash",
            "api_key": food_api_key,
            "base_url": "https://generativelanguage.googleapis.com/v1beta/openai/",
            "client": None  # Inicializado posteriormente
        },
        "activity": {
            "model": "gemini-2.0-flash",
            "api_key": activities_api_key,
            "base_url": "https://generativelanguage.googleapis.com/v1beta/openai/",
            "client": None  # Inicializado posteriormente
        },
        "spotify": {
            "model": "gemini-2.0-flash",
            "api_key": spotify_api_key,
            "base_url": "https://generativelanguage.googleapis.com/v1beta/openai/",
            "client": None  # Inicializado posteriormente
        },
        "party_place": {
            "model": "gemini-2.0-flash",
            "api_key": party_place_api_key,
            "base_url": "https://generativelanguage.googleapis.com/v1beta/openai/",
            "client": None  # Inicializado posteriormente
        }
    }
    try:
        gemini_api = OpenAI(
            base_url=api_configurations["gemini"]["url"],
            api_key=api_configurations["gemini"]["api_key"],
        )
        print("Modelo Gemini configurado com sucesso.")
    except Exception as e:
        print(f"Erro ao configurar o modelo Gemini: {e}")
        gemini = None  # Garante que gemini seja None em caso de falha
    try:
        food_api = OpenAI(api_key=api_configurations["food"]["api_key"], base_url=api_configurations["food"]["base_url"])
        activities_api = OpenAI(api_key=api_configurations["activity"]["api_key"], base_url=api_configurations["activity"]["base_url"])
        spotify_api = OpenAI(api_key=api_configurations["spotify"]["api_key"], base_url=api_configurations["spotify"]["base_url"])
        party_place_api = OpenAI(api_key=api_configurations["party_place"]["api_key"], base_url=api_configurations["party_place"]["base_url"])
        print("Outras APIs configuradas com sucesso.")
    except Exception as e:
        print(f"Erro ao configurar as APIs: {e}")
        # Considerar definir as APIs como None em caso de falha, similar ao Gemini
        food_api = None
        activities_api = None
        spotify_api = None
        party_place_api = None
        print("Outras APIs configuradas com sucesso.")
except Exception as e:
    print(f"Erro ao configurar as APIs: {e}")
    gemini = None  # Impede a execução se o modelo não estiver configurado

DB_FILE = "event_data_poc.json"

def read_db():
    """Lê os dados do ficheiro JSON."""
    if not os.path.exists(DB_FILE):
        return {"eventos_registados": []}
    try:
        with open(DB_FILE, "r", encoding="utf-8") as f:
            # Lidar com ficheiro vazio
            content = f.read()
            if not content:
                return {"eventos_registados": []}
            return json.loads(content)
    except (json.JSONDecodeError, FileNotFoundError):
        return {"eventos_registados": []}  # Retorna uma estrutura válida em caso de erro

def write_db(data):
    """Escreve os dados no ficheiro JSON."""
    with open(DB_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def get_food_suggestions(event_details, prompt):
    """Obtém sugestões de comida."""
    try:
        completion = food_api.chat.completions.create(
            model=api_configurations["food"]["model"],
            messages=[{"role": "user", "content": f"{prompt}. Detalhes do Evento: {event_details}"}],
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Erro ao obter sugestões de comida: {e}")
        return "Não foi possível obter sugestões de comida."

def get_activities_suggestions(event_details, prompt):
    """Obtém sugestões de atividades."""
    try:
        completion = activities_api.chat.completions.create(
            model=api_configurations["activity"]["model"],
            messages=[{"role": "user", "content": f"{prompt}. Detalhes do Evento: {event_details}"}],
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Erro ao obter sugestões de atividades: {e}")
        return "Não foi possível obter sugestões de atividades."

def get_spotify_playlist(event_details, prompt):
    """Obtém sugestões de playlist do Spotify."""
    try:
        completion = spotify_api.chat.completions.create(
            model=api_configurations["spotify"]["model"],
            messages=[{"role": "user", "content": f"{prompt}. Detalhes do Evento: {event_details}"}],
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Erro ao obter sugestões de playlist: {e}")
        return "Não foi possível obter sugestões de playlist."

def get_party_place(event_details, prompt):
    """Obtém sugestões de local para a festa."""
    try:
        completion = party_place_api.chat.completions.create(
            model=api_configurations["party_place"]["model"],
            messages=[{"role": "user", "content": f"{prompt}. Detalhes do Evento: {event_details}"}],
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Erro ao obter sugestões de local: {e}")
        return "Não foi possível obter sugestões de local."



    try:
        event_details = request.json
        if not event_details:
            return jsonify({"erro": "Nenhum detalhe do evento foi fornecido."}), 400

        # --- Prompts para cada API ---
        food_prompt = f"Com base nos detalhes do evento, sugira opções de comida. Detalhes do Evento: {event_details}"
        activities_prompt = f"Com base nos detalhes do evento, sugira atividades. Detalhes do Evento: {event_details}"
        spotify_prompt = f"Com base nos detalhes do evento, sugira uma playlist do Spotify. Detalhes do Evento: {event_details}"
        party_place_prompt = f"Com base nos detalhes do evento, sugira um local para a festa. Detalhes do Evento: {event_details}"

        # Obter respostas de várias APIs
        food_response = get_food_suggestions(event_details, food_prompt)
        activities_response = get_activities_suggestions(event_details, activities_prompt)
        spotify_response = get_spotify_playlist(event_details, spotify_prompt)
        party_place_response = get_party_place(event_details, party_place_prompt)

        print(f"Detalhes do evento recebidos para compilação de respostas: {event_details}")

        # --- Prompt para o Gemini compilar as respostas ---
        compilation_prompt = f"""
        Considere as seguintes sugestões para um evento:
        Comida: {food_response}
        Atividades: {activities_response}
        Playlist Spotify: {spotify_response}
        Local da Festa: {party_place_response}

        Com base nestas sugestões, crie uma resposta final em formato Markdown, que inclua um resumo conciso e as sugestões detalhadas.
        """

        try:
            completion = gemini_api.chat.completions.create(
                model=api_configurations["gemini"]["model"],
                messages=[{"role": "user", "content": compilation_prompt}],
            )
            compiled_response = completion.choices[0].message.content
        except Exception as e:
            print(f"Erro ao obter a resposta compilada do Gemini: {e}")
            compiled_response = "Não foi possível compilar as respostas."

        print(f"Resposta compilada pelo Gemini em Markdown: {compiled_response}")

        return jsonify({"compiled_response": compiled_response})

    except Exception as e:
        print(f"Erro inesperado no endpoint /api/compile_responses: {e}")
        return (
            jsonify({"erro": f"Ocorreu um erro interno no servidor: {e}"}),
            500,
        )

Modelo Gemini configurado com sucesso.
Outras APIs configuradas com sucesso.


## 2. Definir Dados de Teste

In [56]:
event_details = {
    "eventName": "Caipirinha",
    "eventType": "Descontração",
    "guestCount": 30,
    "budget": "500 EUR",
    "eventDate": "2025-01-01",
    "eventObjective": "Queremos um ambiente de descontração e diversão.O tema tem a ver com uma bebida da moda associada a bons tempos  "

}

# --- Prompts para cada API --- (Exemplo)
food_prompt = f"Sugira opções de comida e bebida para um evento.E recomenda quantidades em função do numero de participantes, budget e o tema. Detalhes do Evento: {event_details}"
activities_prompt = f"Sugira atividades para um evento. Tem em conta a altura do ano e o budget. Podes também considerar contratar alguam especie de entretenimento Detalhes do Evento: {event_details}"
spotify_prompt = f"Sugira uma playlist do Spotify para um evento.Sugere apenas algumas músicas. Detalhes do Evento: {event_details}"
party_place_prompt = f"Sugira um local para a festa.Tem em conta a altrua do ano com o tempo e o budget. O escritório da empresa que vai organizar o evento tem um terraço espaçoso para cerca de 35 pessoas e uma zona de almoço confortavél para cerca de 15 pessoas.Detalhes do Evento: {event_details}"

## 3. Testar as Funções

In [57]:
print("Food Suggestions:")
food_response = get_food_suggestions(event_details, food_prompt)
print(food_response)


Food Suggestions:
## Sugestões para o Evento "Caipirinha"

Considerando o tema, o objetivo de descontração e o orçamento de 500 EUR para 30 pessoas, aqui estão algumas sugestões de comida e bebida, com quantidades estimadas e dicas para otimizar o orçamento:

**Tema:** "Caipirinha: Bons Tempos" (sugere um ambiente tropical, festivo e nostálgico)

**Bebidas:**

*   **Caipirinha:** A estrela do evento!
    *   **Ingredientes:** Cachaça de boa qualidade, limões frescos, açúcar e gelo.
    *   **Quantidades:**
        *   Cachaça: 6 garrafas (700ml cada) -  aproximadamente 100ml por pessoa (considerando que nem todos beberão caipirinha o tempo todo).
        *   Limões: 5 kg (aproximadamente 150 limões) - 5 limões por garrafa de cachaça.
        *   Açúcar: 2 kg
        *   Gelo: 15 kg (para garantir que não falte)
    *   **Variações:**
        *   Ofereça opções de frutas para caipirinhas diferentes: morango, maracujá, kiwi. Compre frutas da época para economizar.
        *   Caipirinha 

In [58]:
print("Activities Suggestions:")

activities_response = get_activities_suggestions(event_details, activities_prompt)


print(activities_response)

Activities Suggestions:
## Sugestões de Atividades e Entretenimento para o Evento "Caipirinha" (Janeiro, Orçamento 500€)

Considerando que o evento é em Janeiro e o objetivo é descontração e diversão com um tema "Caipirinha", aqui estão algumas ideias, levando em conta o orçamento limitado:

**Atividades e Decoração Temática (Prioridade):**

*   **Bar de Caipirinhas DIY:**
    *   **Ideia:** Crie um espaço onde os convidados possam fazer as suas próprias caipirinhas.
    *   **Como:** Forneça todos os ingredientes (cachaça, limões, açúcar, gelo, frutas extras como morango, maracujá, kiwi) e utensílios. Tenha algumas receitas simples impressas para inspiração.
    *   **Vantagem:** Interativo, divertido e ajuda a controlar o consumo (e o custo).
*   **Decoração:**
    *   **Simples e Eficaz:** Use cores vibrantes como verde, amarelo e branco. Toalhas de mesa coloridas, balões, fitas e guarda-sóis de papel podem criar um ambiente festivo sem gastar muito.
    *   **Detalhes Temáticos:** 

In [59]:

spotify_response = get_spotify_playlist(event_details, spotify_prompt)
print(spotify_response)



## Sugestão de Playlist "Caipirinha Vibes" para o seu evento:

Considerando o tema "Caipirinha", o ambiente de descontração e a vibe de "bons tempos", sugiro uma playlist com foco em músicas brasileiras animadas e alguns hits internacionais que combinem com o clima leve e divertido.

**Algumas sugestões:**

1.  **"Mas Que Nada" - Sérgio Mendes:** Clássico instantâneo para começar a festa com energia brasileira.
2.  **"Aquarela do Brasil" - Vários artistas:** Uma ode ao Brasil que todo mundo conhece e canta junto.
3.  **"Garota de Ipanema" - Stan Getz & João Gilberto:** Para um toque de bossa nova elegante e relaxante.
4.  **"Não Quero Dinheiro (Só Quero Amar)" - Tim Maia:** Impossível não cantar e dançar com Tim Maia.
5.  **"Evidências" - Chitãozinho e Xororó:** Um clássico sertanejo para embalar os corações e criar um momento divertido de karaokê improvisado.
6.  **"September" - Earth, Wind & Fire:** Um hit disco que levanta qualquer festa e remete a bons tempos.
7.  **"Happy" - Pharr

In [60]:


party_place_response = get_party_place(event_details, party_place_prompt)
print(party_place_response)

Considerando os detalhes do evento "Caipirinha", o orçamento limitado de 500 EUR e a data em Janeiro, sugiro utilizar o próprio escritório da empresa, aproveitando o terraço espaçoso e a zona de almoço.

**Justificativa:**

*   **Orçamento:** A utilização do espaço já existente elimina custos significativos com aluguel de espaço, permitindo que o orçamento seja direcionado para outros elementos cruciais, como bebidas, comida e decoração temática.
*   **Clima em Janeiro:** Janeiro pode ser imprevisível. Ter o terraço como opção principal, com a zona de almoço confortável como alternativa coberta, garante flexibilidade caso o tempo não esteja favorável.
*   **Capacidade:** O terraço acomoda o número de convidados (30) confortavelmente, e a zona de almoço pode ser usada para servir a comida e bebidas, ou como um espaço de apoio.
*   **Tema "Caipirinha":** O ambiente descontraído pode ser facilmente criado com decorações simples e coloridas que remetam ao Brasil, à caipirinha e ao verão.



In [63]:
compilation_prompt = f"""
        Considere as seguintes sugestões para um evento que outros agentes decidiram e tu vais servir como compilador de todos para apresentar uma resposta unidficada e coerente:
        Comida: {food_response}
        Atividades: {activities_response}
        Playlist Spotify: {spotify_response}
        Local da Festa: {party_place_response}

        Com base nestas sugestões, crie uma resposta final em formato Markdown.
        """

try:
            completion = gemini_api.chat.completions.create(
                model=api_configurations["gemini"]["model"],
                messages=[{"role": "user", "content": compilation_prompt}],
            )
            compiled_response = completion.choices[0].message.content
except Exception as e:
            print(f"Erro ao obter a resposta compilada do Gemini: {e}")
            compiled_response = "Não foi possível compilar as respostas."

print(f"Resposta compilada pelo Gemini em Markdown: \n{compiled_response}")

Resposta compilada pelo Gemini em Markdown: 
## Evento "Caipirinha: Bons Tempos" - Resumo Unificado

Este documento consolida as sugestões para o evento "Caipirinha: Bons Tempos", detalhando comida, bebida, atividades, playlist e local, otimizando o orçamento de 500 EUR para 30 pessoas em um evento descontraído e temático em Janeiro.

**Tema:** "Caipirinha: Bons Tempos" (Ambiente tropical, festivo e nostálgico)

**Local:** Escritório da empresa, utilizando o terraço e a zona de almoço. A escolha visa eliminar custos de aluguel e oferecer flexibilidade em caso de mau tempo.

*   **Terraço:** Decoração com luzes pisca-pisca, bandeirinhas coloridas, toalhas de mesa tropicais e plantas. Mobiliário existente com adição de espreguiçadeiras/puffs (se possível).
*   **Zona de Almoço:** Bar improvisado para caipirinhas, buffet de comida e decoração temática.

**Comida e Bebidas (Orçamento Estimado: 435 EUR - Ajustar conforme compras)**

*   **Bebidas:**
    *   **Caipirinha (DIY):**
        *  

## 4. Próximos Passos
 - **Modificar os Dados de Teste:** Altere os `event_details` e os `prompts` para testar diferentes cenários.
 - **Verificar as Saídas:** Analise as saídas das funções para verificar se estão corretas e se as APIs estão a funcionar como esperado.
 - **Debug:** Use o debugger do Jupyter Notebook para inspecionar as variáveis e entender o fluxo do código.